In [1]:
# trazendo a base que foi respondido de ura
import pymysql
import pandas as pd 
import numpy as np
import glob  
import warnings
warnings.filterwarnings('ignore')

# Estabelece a conexão com o banco de dados
connection = pymysql.connect(
    host='10.0.6.2',
    user='planejamento',
    password='@Planejamento!123',
    database='atn'
)

# Define a consulta SQL
query = """
SELECT
DATE_FORMAT(a.instante, '%d-%m') AS DATA
,a.Chave1 AS DEVEDOR
,a.Chave2 AS NOME
,a.Chave3 AS CPF
,CONCAT(a.DDD, a.Fone) AS telefone
,a.id_arquivo
,d.ARC_DESC
FROM totalinfo_2023_07 a 
	LEFT JOIN tabulacaooper top ON a.callid = top.callid
	LEFT JOIN arquivo_mailing d ON a.id_arquivo = d.ID_ARQUIVOMAILING
WHERE a.GrupoPrincipal IN(6189,6187,6191)
AND a.Agente<>''
AND top.codtabulacao <> 50
AND a.instante>='2023-07-04 08:00:00'
AND a.instante<='2023-07-04 23:00:00'
"""

respondidos_claro_net = pd.read_sql(query, connection)

respondidos_claro_net['ARC_DESC'] = respondidos_claro_net['ARC_DESC'].str.upper()
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('NET'), 'CARTEIRA'] = 'NET'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('FIXO'), 'CARTEIRA'] = 'FIXO'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('MOVEL'), 'CARTEIRA'] = 'MOVEL'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('TV'), 'CARTEIRA'] = 'TV'
respondidos_claro_net.loc[respondidos_claro_net['ARC_DESC'].str.contains('GEVENUE'), 'CARTEIRA'] = 'GEVENUE'

respondidos_gevenue = respondidos_claro_net.query("CARTEIRA == 'GEVENUE'")


respondidos_claro_net = respondidos_claro_net.loc[:,['CPF','NOME','telefone','CARTEIRA']]

respondidos_movel = respondidos_claro_net.query("CARTEIRA == 'MOVEL'")
respondidos_tv = respondidos_claro_net.query("CARTEIRA == 'TV'")
respondidos_fixo = respondidos_claro_net.query("CARTEIRA == 'FIXO'")
respondidos_net = respondidos_claro_net.query("CARTEIRA == 'NET'")


#DANDO ALEATORIO NOS ARQUIVOS
respondidos_movel = respondidos_movel.sample(frac = 1)
respondidos_tv = respondidos_tv.sample(frac = 1)
respondidos_fixo = respondidos_fixo.sample(frac = 1)


respondidos_gevenue = respondidos_gevenue.loc[:,['DEVEDOR','NOME','telefone','CARTEIRA']]
respondidos_gevenue = respondidos_gevenue.sample(frac = 1)


In [2]:
### baixando os dfs
respondidos_movel.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Julho\5\Jorgin da Clock/respondidos_cpc_movel_0507.csv",index=False,sep=';')
respondidos_tv.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Julho\5\Jorgin da Clock/respondidos_cpc_tv_0507.csv",index=False,sep=';')
respondidos_fixo.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Julho\5\Jorgin da Clock/respondido_cpc_fixo_0507.csv",index=False,sep=';')
respondidos_net.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Julho\5\Jorgin da Clock/respondidos_cpc_net_0507.csv",index=False,sep=';')
respondidos_gevenue.to_csv(r"R:\TI\TELEFONIA\Mailings Prontos\2023\Julho\5\Jorgin da Clock/respondidos_cpc_gevenue_0507.csv",index=False,sep=';')